In [1]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

In [2]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

In [3]:
train = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/train/", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 3360 files belonging to 6 classes.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Found 720 files belonging to 6 classes.
Found 720 files belonging to 6 classes.


2022-08-01 10:01:28.870783: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-01 10:01:28.870903: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# data_augmentation = keras.Sequential(
# [
#     layers.RandomFlip("horizontal"),
#     layers.RandomRotation(0.1),
#     layers.RandomZoom(0.2),
#     layers.RandomContrast(0.2),
#     layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
# ] 
# )

In [5]:
inputs = keras.Input(shape=(100, 100, 3))

# x = data_augmentation(inputs)
x = layers.Conv2D(16, 5, activation='relu')(inputs)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(16, 5, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.MaxPooling2D(2)(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Flatten()(x)

x = layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.008), activation='relu')(x)

x = layers.Dropout(0.3)(x)

outputs = layers.Dense(6, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 96, 96, 16)        1216      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 16)        6416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 22, 22, 16)       0         
 2D)                                                             
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 11, 11, 16)       0     

In [6]:
callbacks_list = [
    # keras.callbacks.EarlyStopping(
    #     monitor="val_accuracy",
    #     patience=7
    # ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/zenodo_5.h5",
        monitor="val_loss",
        save_best_only=True
    )
]

In [7]:
history = model.fit(
  train.cache(),
  epochs=40,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/40


2022-08-01 10:01:29.448658: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-01 10:01:29.448733: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - ETA: 0s - loss: 4.5482 - accuracy: 0.1583

2022-08-01 10:01:32.218750: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 3s 28ms/step - loss: 4.5482 - accuracy: 0.1583 - val_loss: 1.9465 - val_accuracy: 0.1667
Epoch 2/40
105/105 [==============================] - 3s 24ms/step - loss: 2.4087 - accuracy: 0.1595 - val_loss: 1.8724 - val_accuracy: 0.1667
Epoch 3/40
105/105 [==============================] - 2s 23ms/step - loss: 1.9255 - accuracy: 0.1533 - val_loss: 1.8208 - val_accuracy: 0.1667
Epoch 4/40
105/105 [==============================] - 2s 23ms/step - loss: 1.8617 - accuracy: 0.2759 - val_loss: 1.4673 - val_accuracy: 0.4028
Epoch 5/40
105/105 [==============================] - 2s 23ms/step - loss: 1.4576 - accuracy: 0.4500 - val_loss: 1.0050 - val_accuracy: 0.6611
Epoch 6/40
105/105 [==============================] - 2s 23ms/step - loss: 1.0557 - accuracy: 0.5848 - val_loss: 0.6823 - val_accuracy: 0.7569
Epoch 7/40
105/105 [==============================] - 2s 23ms/step - loss: 0.8543 - accuracy: 0.6661 - val_loss: 0.6700 - val_accuracy: 0.7708
Epoch 8/40

In [8]:
model.evaluate(test)

23/23 [==============================] - 0s 13ms/step - loss: 0.9069 - accuracy: 0.8736


[0.9068769812583923, 0.8736111521720886]

In [9]:
# keras.models.save_model(model, "./models/zenodo_3.h5")

load_model = keras.models.load_model("models/zenodo_5.h5")
load_model.evaluate(test)

 9/23 [==========>...................] - ETA: 0s - loss: 0.5350 - accuracy: 0.8646

2022-08-01 10:03:09.876746: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 0s 13ms/step - loss: 0.5746 - accuracy: 0.8583


[0.5746487975120544, 0.8583333492279053]